In [13]:
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import yaml

method = 'LR'


method = "MICE"

method = 'KNN'
method = 'MF'
gap_type = 'continuous'
p = 35 # gap length
q = 24 # number of missing sites

with open(f'gaps_{gap_type}_{p}_{q}.yaml') as file:
    metadata = yaml.safe_load(file)

gaps = metadata['gaps']
#print(metadata)
cols = list(gaps.keys())

suffix = f'_{gap_type}_{p}_{q}_{method}'
df_imputed = pd.read_csv(f'sim{suffix}.csv', parse_dates=True, index_col=0)
df = pd.read_csv(f'obs{suffix}.csv', parse_dates=True, index_col=0)

# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Data Visualization with Column Selector"),
    dcc.Dropdown(
        id='column-selector',
        options=[{'label': col, 'value': col} for col in cols],
        value = cols[0],
     #   value = 'S_299',
        #options=[{'label': col, 'value': col} for col in df.columns if col != 'time'],
        #value=df.columns[49]  # Default column
    ),
    dcc.Graph(id='line-plot')
])

@app.callback(
    Output('line-plot', 'figure'),
    [Input('column-selector', 'value')]
)
def update_plot(selected_column):
    fig = px.line(
        df,
        y=selected_column,
        title=f"Temperature at Site {selected_column}"
        
    )

    fig.add_scatter(x=df_imputed.index, 
                    y=df_imputed[selected_column], mode='lines', name = 'Imputed')

    if selected_column in gaps:
        fig.add_shape(
            type="rect",
            x0=df.index[gaps[selected_column][0]],
            x1=df.index[gaps[selected_column][1]],
            y0=df_imputed[selected_column].min(skipna=True),
            y1=df_imputed[selected_column].max(skipna=True),
            fillcolor="lightgreen",
            opacity=0.8,
            layer="below",
            line_width=0,
        )

    return fig

if __name__ == '__main__':
    app.run(debug=True)